In [1]:
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt


In [2]:
def kf_pixel_update(ykp1, xkp1_pred):
    # Update
    xkp1_corrected = (np.eye(len(LC))-LC)@xkp1_pred + L@ykp1 + d

    return xkp1_corrected

Load data

In [3]:
start_ind = 20
end_ind = 1000

In [4]:
data = np.load("data/cartpole_train_data_ten_minutes.npz")

In [5]:
measurements = data["measurements"][start_ind:end_ind, :]
pixels = data["frames"][start_ind:end_ind, :]
est_states = data["est_states"][start_ind:end_ind, :]
predicted_states = data["est_states_predicted"][start_ind:end_ind, :]
controls = data["filtered_controls"][start_ind:end_ind, :]

In [6]:
pixel_mat = np.array(pixels.transpose(1, 2, 0).reshape(-1, np.shape(pixels)[0]))
est_state_mat = est_states.T
predicted_state_mat = predicted_states.T
control_mat = controls.T
bias_mat = np.ones((1, np.shape(pixel_mat)[1]))

Set up LLS problem

In [7]:
rhs_mat = est_state_mat - predicted_state_mat
lhs_mat = np.vstack((-predicted_state_mat[:2, :], pixel_mat, bias_mat))
lhs_mat_test = np.vstack((-predicted_state_mat, pixel_mat, bias_mat))

Solve LLS problem

In [8]:
sol = np.linalg.lstsq(lhs_mat.T, rhs_mat.T, rcond=None)[0].T

Extract KF matrices

In [9]:
LC = np.hstack([sol[:, :2], np.zeros((4, 2))])
L = sol[:, 2:-1]
d = sol[:, -1]

Save matrices

In [10]:
np.savez("data/learned_kf_matrices.npz", LC=LC, L=L, d=d)

Test KF matrices

In [11]:
sol_mat_test = np.hstack([LC, L, d.reshape(-1, 1)])
print(np.linalg.norm(rhs_mat - sol_mat_test @ lhs_mat_test))


1.4610676467593713e-12


In [12]:
print(np.linalg.norm(est_states[start_ind] - kf_pixel_update(pixels[start_ind].reshape(-1), predicted_states[start_ind])))

4.6321240020525116e-14


In [13]:
print(kf_pixel_update(pixels[start_ind].reshape(-1), predicted_states[start_ind]))

[-0.02020514  0.00306068  0.0360523  -0.00573253]


In [14]:
print(est_states[start_ind])

[-0.02020514  0.00306068  0.0360523  -0.00573253]
